In [ ]:
# Import necessary libraries
import pandas as pd
import re

# Read CSV
df = pd.read_csv(r"../data/RWventas.csv")

# 1. Clean Special Characters (@, #, *, etc.)
def clean_special_characters(text):
    """Removes special characters from a text"""
    if pd.isna(text):
        return text
    if isinstance(text, str):
        # Remove common special characters
        text = re.sub(r'[@#*$%^&(){}[\]<>~`]', '', text)
        # Remove extra spaces
        text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply cleaning to all text columns
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].apply(clean_special_characters)

# 2. REORDER COLUMNS TO THE CORRECT ORDER
current_columns = df.columns.tolist()
print(f"Detected original columns: {current_columns}")

# Define expected correct column order
correct_columns = [
    'Ciudad', 'Fecha', 'Producto', 'Tipo_Producto', 'Cantidad', 
    'Precio_Unitario', 'Tipo_Venta', 'Tipo_Cliente', 'Descuento', 
    'Costo_Envio', 'Pais'
]

# Create DataFrame with columns in correct order
if len(df.columns) == len(correct_columns):
    df_new = pd.DataFrame()
    for i, correct_name in enumerate(correct_columns):
        df_new[correct_name] = df.iloc[:, i]
    df = df_new
    print(" Columns successfully reordered")
else:
    print(f" Warning: Column count mismatch. Expected: {len(correct_columns)}, Found: {len(df.columns)}")

# Final column order
final_order = [
    'Fecha', 'Producto', 'Tipo_Producto', 'Cantidad', 
    'Precio_Unitario', 'Ciudad', 'Pais', 'Tipo_Venta', 
    'Tipo_Cliente', 'Descuento', 'Costo_Envio'
]

columns_present = [col for col in final_order if col in df.columns]
df = df[columns_present]

# RENAME PAIS COLUMN TO COSTO_TOTAL
if 'Pais' in df.columns:
    df = df.rename(columns={'Pais': 'Costo_Total'})
    print(" Column 'Pais' renamed to 'Costo_Total'")

# Reorder with updated column name
updated_final_order = [
    'Fecha', 'Producto', 'Tipo_Producto', 'Cantidad', 
    'Precio_Unitario', 'Ciudad', 'Tipo_Venta', 
    'Tipo_Cliente', 'Descuento', 'Costo_Envio', 'Costo_Total'
]

columns_present_final = [col for col in updated_final_order if col in df.columns]
df = df[columns_present_final]

# 3. STANDARDIZE TEXT CASE FORMAT
if 'Ciudad' in df.columns:
    df['Ciudad'] = df['Ciudad'].str.strip().str.title()

if 'Producto' in df.columns:
    df['Producto'] = df['Producto'].str.strip().str.capitalize()

categorical_columns = ['Tipo_Producto', 'Tipo_Venta', 'Tipo_Cliente']
for col in categorical_columns:
    if col in df.columns:
        df[col] = df[col].str.strip().str.title().str.replace('_', ' ')

# 4. FIX KNOWN TYPOGRAPHICAL ERRORS
product_corrections = {
    'Yogur': 'Yogurt',
    'Yogurth': 'Yogurt',
    'yogurtt': 'Yogurt',
    'Gasosa': 'Gaseosa',
    'Mantequila': 'Mantequilla',
    'te': 'Té',
    'Galleta': 'Galletas',
    'galletass': 'Galletas',
    'Chocolates': 'Chocolate'
}

type_product_corrections = {
    'Alimento Percedero': 'Alimento Perecedero',
    'Alimento Perecedro': 'Alimento Perecedero',
    'Lacteo': 'Lácteo',
    'Lacteos': 'Lácteo',
    'Abrrotes': 'Abarrotes',
    'Abarotes': 'Abarrotes'
}

sales_type_corrections = {
    'Tienda Fisica': 'Tienda Física',
    'Distribudor': 'Distribuidor'
}

client_type_corrections = {
    'Mayorita': 'Mayorista',
    'Mayorsita': 'Mayorista',
    'Govierno': 'Gobierno'
}

# Apply corrections
if 'Producto' in df.columns:
    for wrong, correct in product_corrections.items():
        df['Producto'] = df['Producto'].replace(wrong, correct, regex=False)

if 'Tipo_Producto' in df.columns:
    for wrong, correct in type_product_corrections.items():
        df['Tipo_Producto'] = df['Tipo_Producto'].replace(wrong, correct, regex=False)

if 'Tipo_Venta' in df.columns:
    for wrong, correct in sales_type_corrections.items():
        df['Tipo_Venta'] = df['Tipo_Venta'].replace(wrong, correct, regex=False)

if 'Tipo_Cliente' in df.columns:
    for wrong, correct in client_type_corrections.items():
        df['Tipo_Cliente'] = df['Tipo_Cliente'].replace(wrong, correct, regex=False)

# Fix Tipo_Producto based on Producto
mapping_type_product = {
    'Arepa': 'Alimento Perecedero',
    'Cafe': 'Bebida',
    'Café': 'Bebida',
    'Chocolate': 'Snack',
    'Galletas': 'Snack',
    'Gaseosa': 'Bebida',
    'Leche': 'Lácteo',
    'Mantequilla': 'Lácteo',
    'Pan': 'Abarrotes',
    'Queso': 'Lácteo',
    'Té': 'Bebida',
    'Yogurt': 'Lácteo',
    'Cereal': 'Abarrotes'
}

def assign_product_type(row):
    if pd.notna(row['Producto']):
        product = row['Producto'].strip().title()
        for prod, type_p in mapping_type_product.items():
            if product.lower() == prod.lower():
                return type_p
    return row['Tipo_Producto'] if pd.notna(row['Tipo_Producto']) else pd.NA

if 'Producto' in df.columns and 'Tipo_Producto' in df.columns:
    df['Tipo_Producto'] = df.apply(assign_product_type, axis=1)
    print(" Product types corrected according to product name")

# 5. CLEAN NULLS AND INVALID VALUES
null_values = ['???', '', 'NA', 'N/A', 'null', 'NULL', 'None', '-']
for value in null_values:
    df = df.replace(value, pd.NA)

df = df.replace(r'^\s*$', pd.NA, regex=True)

# 6. DATA TYPE CONVERSION
numeric_columns = ['Cantidad', 'Precio_Unitario', 'Descuento', 'Costo_Envio']
for col in numeric_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

if 'Costo_Total' in df.columns:
    df['Costo_Total'] = pd.to_numeric(df['Costo_Total'], errors='coerce')

if 'Fecha' in df.columns:
    df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce', format='%Y-%m-%d')

# Trim spaces again
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.strip()

# Drop empty rows
df = df.dropna(how='all')

# Drop rows missing critical data
critical_columns = ['Fecha', 'Producto', 'Cantidad', 'Precio_Unitario']
critical_present = [col for col in critical_columns if col in df.columns]
if critical_present:
    df = df.dropna(subset=critical_present, how='any')

# Fill values in key columns
if 'Ciudad' in df.columns:
    df['Ciudad'] = df['Ciudad'].fillna('Unknown')

if 'Tipo_Venta' in df.columns:
    df['Tipo_Venta'] = df['Tipo_Venta'].fillna('Unknown')

if 'Tipo_Cliente' in df.columns:
    df['Tipo_Cliente'] = df['Tipo_Cliente'].fillna('Unknown')

# Reset index
df = df.reset_index(drop=True)

# Summary
print("=" * 60)
print("DATA CLEANING SUMMARY")
print("=" * 60)
print(f"\nRows after cleaning: {len(df)}")
print(f"\nColumns: {list(df.columns)}")
print("\nNull values per column:")
print(df.isnull().sum())
print("\nData types:")
print(df.dtypes)

print("\n" + "=" * 60)
print("FIRST 20 ROWS OF CLEAN DATASET:")
print("=" * 60)
print(df.head(20).to_string())

# Save cleaned CSV
df.to_csv('clean_sales.csv', index=False, encoding='utf-8-sig')
print("\n File saved as 'clean_sales.csv'")

print("\n" + "=" * 60)
print("DESCRIPTIVE STATISTICS:")
print("=" * 60)
print(df.describe())


Detected original columns: ['Ciudad', 'Fecha', 'Producto', 'Tipo_Producto', 'Cantidad', 'Precio_Unitario', 'Tipo_Venta', 'Tipo_Cliente', 'Descuento', 'Costo_Envio', 'Total']
✓ Columns successfully reordered
✓ Column 'Pais' renamed to 'Costo_Total'
✓ Product types corrected according to product name
DATA CLEANING SUMMARY

Rows after cleaning: 975363

Columns: ['Fecha', 'Producto', 'Tipo_Producto', 'Cantidad', 'Precio_Unitario', 'Ciudad', 'Tipo_Venta', 'Tipo_Cliente', 'Descuento', 'Costo_Envio', 'Costo_Total']

Null values per column:
Fecha                 0
Producto              0
Tipo_Producto         0
Cantidad              0
Precio_Unitario       0
Ciudad                0
Tipo_Venta            0
Tipo_Cliente          0
Descuento          7657
Costo_Envio        7624
Costo_Total        7608
dtype: int64

Data types:
Fecha              datetime64[ns]
Producto                   object
Tipo_Producto              object
Cantidad                  float64
Precio_Unitario           float64
C